In [106]:
import pandas as pd
import os
import re

# Define paths
original_file_path = "/home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/games.csv"
new_folder_path = "/home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data"
new_file_path = os.path.join(new_folder_path, "games_cleaned.csv")

# Columns to keep
columns_to_keep = ["app_id", "name", "release_date", "is_free", "price_overview", "languages", "type"]

# Create new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Read and filter the CSV
df = pd.read_csv(original_file_path, usecols=columns_to_keep)

# Remove rows where 'is_free' is 1 or '1'
df = df[~df["is_free"].isin([1, "1"])]
df.rename(columns={"languages": "initial_price"}, inplace=True)


# Save the cleaned CSV
df.to_csv(new_file_path, index=False)

print(f"Cleaned CSV saved to: {new_file_path}")


/tmp/ipykernel_5165/1960683913.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(original_file_path, usecols=columns_to_keep)


Cleaned CSV saved to: /home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/games_cleaned.csv


In [ ]:
# Load CSV
file_path = "/home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/games_cleaned.csv"
df = pd.read_csv(file_path)

# Clean 'price_overview'
df["price_overview"] = df["price_overview"].astype(str).str.replace(r'\{\\?final\\?"?:\s*', '', regex=True)
df["price_overview"] = df["price_overview"].str.replace(r',.*', '', regex=True)

# Clean 'initial_price'
df["initial_price"] = df["initial_price"].astype(str).str.replace(r'"?\\?\s*initial\\?"?:\s*', '', regex=True)
df["initial_price"] = df["initial_price"].str.replace(r',.*', '', regex=True)
df["initial_price"] = df["initial_price"].str.replace(r'\\', '', regex=True)  # Remove leftover backslashes

# Save cleaned data
df.to_csv(file_path, index=False)

print("Fully cleaned 'price_overview' and 'initial_price' columns.")


✅ Fully cleaned 'price_overview' and 'initial_price' columns.


In [108]:
# Load app_ids from cleaned games file as strings
games_df = pd.read_csv(os.path.join(cleaned_path, "games_cleaned.csv"))
valid_app_ids = set(games_df["app_id"].astype(str))

# Process each file
for file_name in files_to_clean:
    file_path = os.path.join(base_path, file_name)
    
    # Read the CSV with error tolerance
    df = pd.read_csv(file_path, on_bad_lines='skip', engine='python')
    
    # Convert app_id to string before filtering
    if "app_id" in df.columns:
        df["app_id"] = df["app_id"].astype(str)
        df_cleaned = df[df["app_id"].isin(valid_app_ids)]
    else:
        print(f"'app_id' column not found in {file_name}")
        continue

    # Save cleaned file
    cleaned_file_path = os.path.join(cleaned_path, f"{file_name.replace('.csv', '_cleaned.csv')}")
    df_cleaned.to_csv(cleaned_file_path, index=False)
    print(f"Cleaned {file_name} saved to {cleaned_file_path} with {len(df_cleaned)} rows")

Cleaned genres.csv saved to /home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/genres_cleaned.csv with 285703 rows
Cleaned reviews.csv saved to /home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/reviews_cleaned.csv with 106365 rows
Cleaned steamspy_insights.csv saved to /home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/steamspy_insights_cleaned.csv with 106326 rows
Cleaned tags.csv saved to /home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/tags_cleaned.csv with 106421 rows


In [110]:
file_path = "/home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/games_cleaned.csv"
df = pd.read_csv(file_path)

# Clean any non-digit characters from the price columns
df["price_overview"] = df["price_overview"].astype(str).str.extract(r'(\d+)')[0]
df["initial_price"] = df["initial_price"].astype(str).str.extract(r'(\d+)')[0]

# Convert to numeric and then to dollars
df["price_overview"] = pd.to_numeric(df["price_overview"], errors='coerce') / 100
df["initial_price"] = pd.to_numeric(df["initial_price"], errors='coerce') / 100

# Round to 2 decimal places
df["price_overview"] = df["price_overview"].round(2)
df["initial_price"] = df["initial_price"].round(2)

# Save final version
df.to_csv(file_path, index=False)

print("Cleaned and converted: 'price_overview' and 'initial_price' are now floats with cents.")

Cleaned and converted: 'price_overview' and 'initial_price' are now floats with cents.


In [111]:
# File path
cleaned_file_path = "/home/cjulias/SteamSalesAndPricingAnalysis/Steam_Project/cleaned_data/games_cleaned.csv"

# Load the CSV
df = pd.read_csv(cleaned_file_path)

# Remove rows where any column contains '\N'
df_cleaned = df[~df.isin(["\\N"]).any(axis=1)]

# Save it back
df_cleaned.to_csv(cleaned_file_path, index=False)
print(f"Removed rows with '\\N'. New shape: {df_cleaned.shape}")

Removed rows with '\N'. New shape: (81362, 7)
